# Assignment 04
## Web Search
## CSCI E-108   

> **Instructions:** For this assignment you will complete the exercises shown. Most exercises involve creating and executing some Python code. Additionally, most exercises have questions for you to answer. You can answer questions by creating a Markdown cell and writing your answer. If you are not familiar with Markdown, you can find a brief tutorial [here](https://www.markdownguide.org/cheat-sheet/).     

In this assignment you will gain some experience and insight into how web search algorithms work. Specifically, you will implement versions of three algorithms, simple PageRank, damped PageRank, and the HITS algorithm. All three of these algorithms use a **directed graph model** of the web.   

The small data examples and coding methods used here are not directly scalable to web sized problems. Rather, the point is for you to understand the basic characteristics of these web search algorithms. Web scale searching requires massive resources not readily available to most people. 

## Simple PageRank Example

To get a feeling for the basics of the PageRank algorithm you will create and test simple code. 

As a first step, execute the code in the cell below to import the packages you will need for the exercises. 

In [169]:
import numpy as np

### Directed Graph of Web Pages

We will start with a simple example. Figure 1 shows a set of web pages and their hyperlinks. This is a **directed graph** with the **pages as nodes** and the **hyperlinks as the directed edges**. This graph is **complete**. Every page is accessible from any other page, possibly with visits to intermediate nodes required.  

<img src="../Web1.png" alt="Drawing" style="width:500px; height:400px"/>
<center>Figure 1: A small set of web pages</center>

The directed edges of the graph define the association between the nodes. For the **association matrix**, a directed edge, or hyperlink, runs from a node's column to the terminal node's row. The association is binary. The directed edge either exists or it does not. 

> **Exercise 04-1:** In the cell below you will create the association matrix and the initial page probability vector. Do the following:  
> 1. Create the association matrix, $A$, using [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html). This matrix is constructed with a 1 where a page in a column has a directed edge linking to another page in a row, and 0 elsewhere. In matrix notation, the element $a_{i,j}$ indicates the presence or absence of a directed edge from node $n_j$ to node $n_i$.   
> 2. Print the shape of your association matrix as a check.
> 3. Print the in degree and out degree of each node in your association matrix, using [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html). Set the argument `axis` to 1 to sum across rows and 0 to sum down columns. 

In [170]:
# Create the association matrix
A = np.array([
    [0, 1, 1, 1, 0],  
    [1, 0, 0, 1, 1],   
    [0, 0, 0, 1, 1],   
    [1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0]
])

print("Shape of association matrix:", A.shape)

# Calculate the in degree and out degree of each node
in_degree = np.sum(A, axis=0)  
out_degree = np.sum(A, axis=1) 

# Print results
print("In-degree of each node:", in_degree)
print("Out-degree of each node:", out_degree)

Shape of association matrix: (5, 5)
In-degree of each node: [2 1 1 4 3]
Out-degree of each node: [3 3 2 2 1]


> Are the out degree and in degree you computed from the association matrix consistent with the graph in Figure 1?    
> **End of exercise.**

> **Answer:**   Yes, they look to be consistent.     

### Apply Simple Page Rank

The normalized transition probability matrix, $M$, is then computed from the association matrix, $A$: 

$$M = A D^{-1}$$

Where, $D^{-1}$ is the inverse of a matrix with the out degree values on the diagonal and zeros elsewhere.  

You can see from the foregoing that $M$ distributes the influence of the page by the in|verse of the out degree. In other words, the influence is inversely weighted by the number of pages each page links to. 

> **Exercise 04-2:** You will now compute the normalized transition matrix, $M$. To do so create a function called `norm_association` with the association matrix as the argument. Do the following: 
> 1. Create your function `norm_association` which will do the following:  
>    - Compute the sum of the columns of the association matrix using `numpy.sum` with the `axis=0` argument to sum along columns. 
>   - Compute the inverse of the column sums as a vector. Be sure to avoid zero divides, which will occur in subsequent exercises. Use the `where` argument of [numpy.divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) to do so. If the column sum is 0 the inverse is set to 0.0.   
>   - Create a square diagonal matrix from the inverse column sums using [numpy.diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html) to form the inverse out degree diagonal matrix.
>  - Finally, return the matrix product of the association matrix and the (diagonal) inverse out degree matrix using [numpy.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).  
> 2. Save and print the normalized transition matrix.  
> 3. Compute and print the column sums of the normalized transition matrix to ensure they all add to 1.0. 
> Execute the code you have created and examine the results. 

In [171]:
def norm_association(A):
    '''Function to normalize the association matrix by out degree.
    The function accounts for cases where the column sum is 0'''
    # Compute the sum of the columns of the association matrix and invert
    column_sums = np.sum(A, axis=1)
    print(column_sums)
    inverse_column_sums = np.divide(1, column_sums, where=column_sums!=0)
    
    # Create a square diagonal matrix from the inverse column sums
    D_inv = np.diag(inverse_column_sums)
    transposed_A = np.transpose(A)
    
    # Return the product of the association matrix and the diagonal inverse column sum matrix
    M = np.matmul(transposed_A, D_inv)

    return M

## Execute the function and check the column sums
M = norm_association(A)
print(M) 
print(f"The column sums = {np.sum(M, axis=0)}")

[3 3 2 2 1]
[[0.         0.33333333 0.         0.5        0.        ]
 [0.33333333 0.         0.         0.         0.        ]
 [0.33333333 0.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.5        0.         1.        ]
 [0.         0.33333333 0.5        0.5        0.        ]]
The column sums = [1. 1. 1. 1. 1.]


> Provide short answers to the following questions:     
> 1. Do the number of non-zero values in each column match the out degree for the corresponding node?     
> 2. Are all the column sums 1.0, and why is this required for a transition probability matrix.    
> **End of exercise.**

> **Answers:**  
> 1. Yes, the number of non-zero values in each column matches the out degree for the corresponding node. Each non-zero value in a column represents an outgoing edge from that node, thus matching its out degree.
  
> 2. Yes, all the column sums are 1.0. This is required for a transition probability matrix because each column represents the probabilities of transitioning from one page (node) to another. Therefore, the sum of probabilities for transitions from a particular page (node) to all other pages (nodes) should be 1.0, ensuring that the system doesn't lose or gain probability mass during transitions.                

### Computing the Simple Page Rank

With the transition probability matrix, $M$ computed it is time to investigate the convergence of the PageRank algorithm. You can think of the PageRank algorithm as a series of transitions of a Markov Chain. Given the transition probability matrix, $M$, the update, or single Markov transition, of the page probabilities, $p_i$, is computed: 

$$p_i = M p_{i-1}$$

The Markov chain can be executed for a great many transitions. The result of $n$ transitions, starting from an initial set of page probabilities, $p_0$, can be written:  

$$p_n = M^n p_{0}$$

At convergence the page probabilities, $p_n$, approach a constant or **steady state** value. This steady state probability vector values are the PageRank of the web pages.  

> **Exercise 04-3:** You will now create and execute code with the goal of getting a feel for how the page probabilities change for a single transition of a Markov process. The accomplish this task you will create a function called `transition` with arguments of the the normalized transition probability matrix and the vector of page probabilities. Specifically you will:   
> 1. Complete the function `transition` which uses [numpy.dot](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) to compute the product of the transition matrix and the page probability vector.  
> 2. Define a state vector p, with uniform initial probabiltes. Print this initial statevector.
> 3. Execute the `transition` function on the normalized transition probability matrix and vector of initial page probabilities you have created, saving the result to a new variable name. Print the result. 
> 4. Print the Euclidean (L2) norm of the difference between the initial page probabilities and the updated page probabilities. 
> 5. Print the sum of the page probabilities computed with `transition`.

In [172]:
def transition(transition_probs, probs):
    '''Function to compute the probabilities resulting from a 
    single transition of a Markov process'''
    # Compute the product of the transition matrix and the page probability vector
    updated_probs = np.dot(transition_probs, probs)
    return updated_probs

# Define a state vector p with uniform initial probabilities
p_initial = np.ones(len(A)) / len(A)
print("Initial state vector:")
print(p_initial)

# Compute probabilities after the first state transition
p_updated = transition(M, p_initial)
print("\nProbabilities after the first state transition:")
print(p_updated)

# Print the norm of the difference between the initial and updated page probabilities
norm_difference = np.linalg.norm(p_initial - p_updated)
print("\nThe norm of the difference = ", f"{norm_difference:.3f}")

# Print the sum of the updated page probabilities
sum_updated_probs = np.sum(p_updated)
print("\nThe sum of the PageRanks = ", sum_updated_probs)

Initial state vector:
[0.2 0.2 0.2 0.2 0.2]

Probabilities after the first state transition:
[0.16666667 0.06666667 0.06666667 0.43333333 0.26666667]

The norm of the difference =  0.309

The sum of the PageRanks =  1.0


> Provide short answers to the following questions:   
> 1. Is the sum of the page probabilities equal to 1.0 as it should be?       
> 2. Considering in degree of the pages, are the relative changes in the page probabilities what you would expect and why?    
> **End of exercise.**

> **Answers:**    
> 1. Yes. Shown above. 
> 2. Yes, the relative changes in the page probabilities are what we would expect based on the in degrees of the pages [2,1,1,4,3] from earlier. Pages  higher in degrees tend to have higher probabilities after the transition, as they receive more influence from other pages linking to them and vice versa. This aligns with the idea that pages with more incoming links are generally considered more important in the PageRank algorithm.    

> **Exercise 04-4:** You will continue with computing transitions of the Markov chain. Use the `transition` function with the normalized transition probability matrix and the page probability vector computed from the first transition as arguments. Your code must do the following:  
> 1. Compute and print the resulting page probabilities of the second transition.
> 2. Compute and print the Euclidean (L2) norm of the difference between the page probabilities before and after the transition. 
> 3. Compute and print the sum of the page probabilities. 
> 4. Display the page probabilities. 

In [173]:
# Compute probabilities after the second state transition
p_updated_second = transition(M, p_updated)
print("\nProbabilities after the second state transition:")
print(p_updated_second)

# Compute the norm of the difference between the page probabilities before and after the transition
norm_difference_second = np.linalg.norm(p_updated - p_updated_second)
print("\n norm of the difference:", f"{norm_difference_second:.3f}")

# Compute the sum of the page probabilities
sum_updated_probs_second = np.sum(p_updated_second)
print("\nSum of the updated page probabilities after the second transition:", sum_updated_probs_second)

# Print results
print("\nPage probabilities after the second transition:")
for i, prob in enumerate(p_updated_second):
    print(f"Page {chr(65+i)}: {prob}")



Probabilities after the second state transition:
[0.23888889 0.05555556 0.05555556 0.37777778 0.27222222]

 norm of the difference: 0.093

Sum of the updated page probabilities after the second transition: 1.0

Page probabilities after the second transition:
Page A: 0.2388888888888889
Page B: 0.05555555555555556
Page C: 0.05555555555555556
Page D: 0.3777777777777778
Page E: 0.27222222222222225


> Note the difference between the Euclidean norms of the differences for the first and second transition calculations. Does the change in this difference from one step to the next indicate the algorithm is converging to the steady state probabilities?  
> **End of exercise.**

> **Answer:**  Yes, in the case of the PageRank algorithm, as the algorithm iterates, the differences between successive probabilities typically decrease, indicating that the algorithm is converging towards a steady-state solution. If the change in the Euclidean norm of the differences from one step to the next becomes smaller with each iteration, it suggests that the algorithm is converging. Once the change becomes sufficiently small, it can be considered that the algorithm has reached convergence, and the page probabilities have stabilized to their steady-state values.

>Therefore, observing a decrease in the difference between successive transition calculations, as indicated by the change in the Euclidean norm, can be a sign that the algorithm is converging to the steady-state probabilities.   

> **Exercise 04-5:** The question now is how does this simplified version of page rank converge with more iterations? To find out, do the following:   
> 1. Create a function `pagerank1` having the following arguments, `the normalized transition matrix`, the `initial page probabilities` and a `convergence threshold value of 0.01`, which does the following:  
>    - Initialize a euclidean distance norm variable to 1.0 and the resulting page probabilities to a vector of 0.0 values of length equal to the dimension of the transition matrix.   
>    - Set a loop counter to 1.  
>    - Use a 'while' loop with termination conditions the euclidean distance norm greater than the threshold value AND the loop counter less than 50.  Inside this loop do the following:  
>      1. Update the page probabilities using the `transition` function you created. 
>      2. Compute the Euclidean norm of the difference between the previous and the updated page probabilities following the transition.   
>      3. Print the value of the loop counter and the Euclidean norm of the difference. 
>      4. Copy the updated page probability vector into the input page probability vector.  
>      5. Increment the loop counter by 1. 
>    - Return the page probabilities at convergence.  
> 2. Execute your `pagerank1` function using the transition matrix and initial page probability vector. 

In [174]:
# Compute probabilities after a number of state transitions 
def pagerank1(M, in_probs,  threshold = 0.01):  
    euclidean_dist = 1.0 
    page_probabilities = np.array([0.0]*len(M))
    i = 1   
    
    while euclidean_dist > threshold and i < 50:
        updated_probs = transition(M, in_probs)
        euclidean_dist = np.linalg.norm(in_probs - updated_probs)
        print(f"Iteration {i}: Euclidean distance norm = {euclidean_dist}")
        in_probs = updated_probs
        i += 1
        
    return updated_probs

# Execute and print results
print('Final state probabilities: ' + str(pagerank1(M, p_initial)))

Iteration 1: Euclidean distance norm = 0.3091206165165235
Iteration 2: Euclidean distance norm = 0.09262962222518367
Iteration 3: Euclidean distance norm = 0.0627447198003608
Iteration 4: Euclidean distance norm = 0.04713034716117627
Iteration 5: Euclidean distance norm = 0.040449830664990756
Iteration 6: Euclidean distance norm = 0.03475146950237423
Iteration 7: Euclidean distance norm = 0.02965169022900905
Iteration 8: Euclidean distance norm = 0.025297173726972603
Iteration 9: Euclidean distance norm = 0.02160299244562213
Iteration 10: Euclidean distance norm = 0.018452365792341104
Iteration 11: Euclidean distance norm = 0.015759786178754207
Iteration 12: Euclidean distance norm = 0.013459509556315244
Iteration 13: Euclidean distance norm = 0.011495031885715202
Iteration 14: Euclidean distance norm = 0.009817345265918275
Final state probabilities: [0.21890828 0.07149265 0.07149265 0.38258726 0.25551917]


> Provide short answers for the following questions:  
> 1. Judged from the rate of decline of the Euclidean distances, does the algorithm appear to converge rapidly and why? 
> 2.  Does the rank order of the computed page probabilities make sense given the relative degree of the pages of the directed graph? 
> **End of exercise.**

> **Answers:**   
> 1. The algorithm appears to converge quite rapidly, this is because the Euclidean distances between successive page probability vectors decrease quickly with each iteration, indicating that the algorithm is approaching convergence to the steady-state probabilities.         
> 2. Yes, the rank order of the computed page probabilities makes sense given the relative degree of the pages of the directed graph. Pages with higher in degrees tend to have higher probabilities, reflecting their importance in the network. Conversely, pages with lower in degrees tend to have lower probabilities. This aligns with the intuition that pages with more incoming links are considered more important in the PageRank algorithm.   

### Page Rank by Eigendecomposition

Consider the relationship:     

$$p_i = M p_{i-1}$$      

At convergence $p_i = p_{i-1}$ which suggest an eigenvalue-eigenvector problem for some eigenvalue, $\lambda$:    

$$\lambda p^* = M p^*$$  

For the transition probability matrix with normalized columns, the largest eigenvalue has a magnitude 1.0. The eigenvector associated with this eigenvalue is the PageRank vector. To demonstrate this point, the code in the cell below does the following:     
1. Compute the eigendecomposition using [numpy.linalg.eig](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html).     
2. Print the magnitude of the eigenvalues.  
3. Get the eigenvector associated with the first, largest, eigenvalue.     
4. Normalize the eigenvector so the values sum to 1.0 and display the results.    

In [175]:
# Compute the eigendecomposition
eigenvalues, eigenvectors = np.linalg.eig(M)
print(eigenvectors)

# Print the magnitude of the eigenvalues
print("Magnitude of eigenvalues:")
print(np.abs(eigenvalues))

# Get the eigenvector associated with the first, largest eigenvalue
largest_eigenvalue_index = np.argmax(np.abs(eigenvalues))
pagerank_vector = eigenvectors[:, largest_eigenvalue_index]
print(pagerank_vector)

# Normalize the eigenvector so the values sum to 1.0
pagerank_vector_normalized = pagerank_vector / np.sum(pagerank_vector)

# Print results
print("\nFinal state probabilities:")
print(pagerank_vector_normalized)


[[-4.17252857e-01+0.j         -4.51280150e-01+0.j
  -6.44849022e-03+0.41375155j -6.44849022e-03-0.41375155j
   1.77796963e-18+0.j        ]
 [-1.39084286e-01+0.j          1.76133214e-01+0.j
   4.30910643e-01-0.09645575j  4.30910643e-01+0.09645575j
  -8.01783726e-01+0.j        ]
 [-1.39084286e-01+0.j          1.76133214e-01+0.j
   4.30910643e-01-0.09645575j  4.30910643e-01+0.09645575j
  -5.03549833e-16+0.j        ]
 [-7.41782856e-01+0.j          6.53410226e-01+0.j
  -5.37666048e-01+0.j         -5.37666048e-01-0.j
   5.34522484e-01+0.j        ]
 [-4.86794999e-01+0.j         -5.54396504e-01+0.j
  -3.17706747e-01-0.22084005j -3.17706747e-01+0.22084005j
   2.67261242e-01+0.j        ]]
Magnitude of eigenvalues:
[1.00000000e+00 8.54050825e-01 3.12368706e-01 3.12368706e-01
 1.25975070e-16]
[-0.41725286+0.j -0.13908429+0.j -0.13908429+0.j -0.74178286+0.j
 -0.486795  +0.j]

Final state probabilities:
[0.21686747-0.j 0.07228916-0.j 0.07228916-0.j 0.38554217-0.j
 0.25301205-0.j]


> **Exercise 04-06:** Do the PageRanks computed by the two methods agree to the precision expected with the iterative method?    
> **End of exercise.**

> **Answer:**   The PageRanks computed by the two methods do agree somewhat to the precision expected with the iterative method. Both methods yield similar PageRank vectors with slight rounding error.           

## A More Complicated Example   

You will now work with a more complicated example The graph of 6 web pages, shown in Figure 2, is no longer complete. The out degree of page 6 is 0. A random surfer transitioning to page 6 will have no escape, a **spider trap**! 

<img src="../Web2.png" alt="Drawing" style="width:500px; height:500px"/>
<center>Figure 3: A small set of web pages with a dead end</center>

> **Exercise 04-7:** You will now create both the normalized transition matrix and the initial page probability vector for the graph of Figure 3. In this exercise you will . Do the following:  
> 1. Create the association matrix and save it to a named variable, `A_deadend`. You will need this association marrix for later 
> 2. Normalize the association matrix using your `norm_association` function. Name your transition matrix `M_deadend`. Print the result. 
> 3. Create a vector containing the uniformly distributed initial probability values. Save and print the result.   

In [176]:
import numpy as np

A_deadend = np.array([
    [0, 1, 1, 1, 0, 1],  
    [1, 0, 0, 1, 1, 0],   
    [0, 0, 0, 1, 1, 1],   
    [1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0]
])

# Normalize the association matrix
M_deadend = norm_association(A_deadend)
print("Normalized transition matrix M_deadend:")
print(M_deadend)

# Create the equal probability starting values
p_initial_deadend = np.ones(len(A_deadend)) / len(A_deadend)
print("\nstarting values:")
print(p_initial_deadend)


[4 3 3 2 1 0]
Normalized transition matrix M_deadend:
[[0.         0.33333333 0.         0.5        0.         0.        ]
 [0.25       0.         0.         0.         0.         0.        ]
 [0.25       0.         0.         0.         0.         0.        ]
 [0.25       0.33333333 0.33333333 0.         1.         0.        ]
 [0.         0.33333333 0.33333333 0.5        0.         0.        ]
 [0.25       0.         0.33333333 0.         0.         0.        ]]

starting values:
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


> Examine your results. Are the 0 values for the transition probabilities of page 6 consistent with the graph of these pages? and why?      
> **End of exercise.**     

> **Answers:** Yes, the 0 values for the transition probabilities of page 6 are consistent with the graph of these pages.
Page 6 has no outgoing edges in the graph, meaning there are no links from page 6 to any other page. This lack of outgoing edges results in 0 transition probabilities for page 6 in the transition matrix.

### Apply Simple PageRank Algorithm

> **Exercises 04-8:** What happens if you apply the simplified PageRank algorithm to the pages on a graph that is not complete, like the one shown in Figure 2? To find out, execute your `pagerank1` function with arguments `M_deadend`, `p_deadend` and `threshold=0.00001`. The smaller threshold value is to ensure convergence. 

In [177]:
# Execute pagerank1 function with given inputs
p_deadend = pagerank1(M_deadend, p_initial_deadend, threshold=0.00001)
print("PageRank probabilities for the non-complete graph:")
print(p_deadend)

Iteration 1: Euclidean distance norm = 0.24689428936987748
Iteration 2: Euclidean distance norm = 0.08729031312483704
Iteration 3: Euclidean distance norm = 0.05112283061425413
Iteration 4: Euclidean distance norm = 0.0409021129345033
Iteration 5: Euclidean distance norm = 0.03330965740599666
Iteration 6: Euclidean distance norm = 0.0315625664858363
Iteration 7: Euclidean distance norm = 0.02559781956037353
Iteration 8: Euclidean distance norm = 0.024309802780145634
Iteration 9: Euclidean distance norm = 0.020108336170986056
Iteration 10: Euclidean distance norm = 0.019211686938674834
Iteration 11: Euclidean distance norm = 0.016185076011934054
Iteration 12: Euclidean distance norm = 0.01548001937133012
Iteration 13: Euclidean distance norm = 0.01326780749245992
Iteration 14: Euclidean distance norm = 0.012663894730572466
Iteration 15: Euclidean distance norm = 0.011017095534036438
Iteration 16: Euclidean distance norm = 0.010475991182690667
Iteration 17: Euclidean distance norm = 0.00

> Now, create and execute code to compute and display the eigenvalues and eigenvectors of the `M_deadend`. Then display the magnitudes of the eigenvalues, and the normalized values of the first eigenvalue (column 0).    

In [178]:
# Compute the eigendecomposition of M_deadend
eigenvalues_deadend, eigenvectors_deadend = np.linalg.eig(M_deadend)

# Display magnitudes of the eigenvalues
print("Magnitudes of the eigenvalues:")
print(np.abs(eigenvalues_deadend))

# Normalize the first eigenvector
first_eigenvector_normalized = eigenvectors_deadend[:, 0] / np.sum(eigenvectors_deadend[:, 0])

# Print results
print("\nNormalized values of the first eigenvalue (column 0):")
print(first_eigenvector_normalized)

print("\nPage probabilities sum:")
print(np.sum(p_deadend))


Magnitudes of the eigenvalues:
[0.00000000e+00 9.20850965e-01 8.17762738e-01 6.65104005e-17
 2.35226392e-01 2.35226392e-01]

Normalized values of the first eigenvalue (column 0):
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]

Page probabilities sum:
0.015476571326304048


> Answer the following questions:  
> 1. Examine the page probabilities computed with the iterative methods. Do these PageRank values sum to 1.0 and why is this outcome a problem?       
> 2. Examine the eigenvalues of the `M_deadend` matrix. What problem can you see with these eigenvalues? 
> 3. Notice the PageRank values have only 0 values. What does this tell you about the convergence of a random surfer on this graph?             
> **End of exercise.**

> **Answers:**   
> 1.  These values sum to ~0.015 and not to 1. This is problematic because PageRank values should represent probabilities, which means they must sum to 1.0 to indicate a valid probability distribution. The deviation from 1.0 indicates a problem in the iterative process, likely caused by the dead-end "spider trap" in the new graph.
> 2.  The presence of an eigenvalue of 0 (the first eigenvalue) indicates that there is at least one absorbing state or dead-end in the graph. This means that once a random surfer reaches this state, there is no way to leave, leading to improper distribution of the probabilities.
>   
>     The other eigenvalues being significantly less than 1 suggest that the system may not properly distribute probabilities across states.
> 
> 3.  We see that all the PageRank values except for one are zero, which means the random surfer will eventually end up in a specific state and stay there indefinitely. This confirms the presence of a dead-end or "absorbing state" in the graph, which traps the random surfer.

### Damped PageRank Algorithm

It is clear from the results of the foregoing exercise that the simple PageRank algorithm does not converge to a usable set of page probabilities when faced with graph that is not complete.Fortunately, there is a simple fix, add a damping term. You can think of the damping term as allowing a random surfer to make an arbitrary transition or jump with some small probability. These random jumps help the random surfer to better explore the graph and to escape from spider traps. The jump probabilities from states, $p_i$, are a function of the damping factor $d$: 

$$Jump\ Probability = \frac{(1-d)}{n}$$

Where $n$ is the dimension of the transition probability matrix. 

The updated page probabilities, $p_i$, are then computed with the damped PageRank algorithm as:   

$$p_{i} = d * M p_{i-1} + \frac{(1-d)}{n}$$

Where $M$ is the transition probability matrix and p are the initial page probability values.   

> **Exercise 04-9:** To implement the PageRank algorithm with a damping factor do the following:  
> 1. Create a `transition_damped` function with arguments, the transition probability matrix, the initial page probabilities, and the damping factor, $d=0.85$, which does the following:  
>   - Compute the updated page probabili|ties by computing the inner (dot) product of the transition probability matrix with the page probabilities and then multiplying by the damping factor, `d`.    
>   - Compute the jump probabilities vector of length the dimension of the transition matrix. Note: the jump probabilities are constant, so you can create code that only computes them once if you so choose.      
>   - Return the sum of the damped page probabilities and the jump probabilities.  
> 2. Create a `pagerank_damped` function. This function is identical to the `pagerank1` function you already created except that it uses the `transiton_damped` function in place of the `transition` function.  
> 3. Call your `pagerank_damped` function using arguments of `M_deadend`, `p_deadend` and `threshold=0.0001` and display the final PageRank vector.
> . Compute and display the sum of the values in the PageRank vector.

In [193]:
## Add a damping factor to the transiton 
def transition_damped(transition_probs, probs, d=0.85):
    '''Function to compute the probabilities resulting from a 
    single transition of a Markov process including a damping
    factor to deal with dead ends'''
    # Compute the updated page probabilities
    updated_probs = d * np.dot(transition_probs, probs)
    
    # Compute the jump probabilities vector
    jump_probs = (1 - d) * np.ones(len(probs)) / len(probs)
    
    # Return sum of damped page probabilities and jump probabilities
    return updated_probs + jump_probs

def pagerank_damped(M, in_probs, d=0.85, threshold=0.01):  
    
    num_iterations = 0
    diff = 1
    page_probabilities = in_probs
    
    # Iterate until convergence
    while diff > threshold:
        next_probs = transition_damped(M, page_probabilities, d)
        diff = np.linalg.norm(next_probs - page_probabilities)
        page_probabilities = next_probs  # Update page_probabilities
        num_iterations += 1
        
        # Print Euclidean distance for each iteration
        print(f"Iteration {num_iterations}: Euclidean distance = {diff}")
    
    return page_probabilities

# Execute the pagerank_damped function
damped_rank = pagerank_damped(M_deadend, p_deadend, d=0.85, threshold=0.0001)
print(f"Sum of the PageRank values = {sum(damped_rank)}")
print('Final state probabilities: ' + str(damped_rank))


Iteration 1: Euclidean distance = 0.059871357485657486
Iteration 2: Euclidean distance = 0.05161018427509011
Iteration 3: Euclidean distance = 0.03897245486729463
Iteration 4: Euclidean distance = 0.030304127611173258
Iteration 5: Euclidean distance = 0.023606004792419994
Iteration 6: Euclidean distance = 0.018605290878110112
Iteration 7: Euclidean distance = 0.014472392398026931
Iteration 8: Euclidean distance = 0.011384898876317596
Iteration 9: Euclidean distance = 0.008869675608674892
Iteration 10: Euclidean distance = 0.006970261239540003
Iteration 11: Euclidean distance = 0.0054357234010308985
Iteration 12: Euclidean distance = 0.004268145263039698
Iteration 13: Euclidean distance = 0.0033311212489827075
Iteration 14: Euclidean distance = 0.0026138807030726447
Iteration 15: Euclidean distance = 0.0020412936114440174
Iteration 16: Euclidean distance = 0.0016009340048835888
Iteration 17: Euclidean distance = 0.0012508459935303823
Iteration 18: Euclidean distance = 0.0009805992629056

> Provide short answers to the following questions:   
> 1. Examine the final page probabilities. Does the rank of these page probabilities make sense given the in degree of the pages of this graph?   
> 2. Why is it reasonable that the sum of the PageRanks is $< 1.0$? 

> **Answers:**     
> 1.  Yes, the final page probabilities make sense given the in degree of the pages of this graph. Pages with higher in-degree tend to have higher PageRank probabilities (e.g. page 4 has the highest), reflecting their importance or popularity within the graph.

> 2. It is reasonable that the sum of the PageRanks (approx ~0.627) is less than 1.0 because of the damping factor (d=0.85) introduced in the damped PageRank algorithm. The damping factor redistributes the PageRank probabilities by allowing random jumps, which reduces the overall influence of the links between pages. As a result, some PageRank probabilities are transferred to the jump probabilities, leading to a sum of PageRanks that is less than 1.0. Additionally, some probability mass may also be lost when jumping to dead-end nodes during transitions, "absorbing" the probability mass without redistributing it.

> Next you will examine the some properties of the damped matrix, $M$. You will do so by the following steps:    
> 4. Create a Numpy array of $M$ including the damping, with a damping factor, $d = 0.85$. Display this matrix.  
> 5. Compute and display the column sums of the damped matrix.  

In [180]:
# Initialization
d = 0.85
M_damped = d * M_deadend + (1 - d) / len(M_deadend)

# Display the damped transition matrix
print("Damped transition matrix M_damped:")
print(M_damped)

# Compute and display column sums of the damped matrix
column_sums_damped = np.sum(M_damped, axis=0)
print("\nColumn sums of the damped matrix:")
print(column_sums_damped)


Damped transition matrix M_damped:
[[0.025      0.30833333 0.025      0.45       0.025      0.025     ]
 [0.2375     0.025      0.025      0.025      0.025      0.025     ]
 [0.2375     0.025      0.025      0.025      0.025      0.025     ]
 [0.2375     0.30833333 0.30833333 0.025      0.875      0.025     ]
 [0.025      0.30833333 0.30833333 0.45       0.025      0.025     ]
 [0.2375     0.025      0.30833333 0.025      0.025      0.025     ]]

Column sums of the damped matrix:
[1.   1.   1.   1.   1.   0.15]


> Examine your results and asnwer these questions:   
> 3. How does this array allow the random surfer to 'teleport' (or transition) from any page to any other page even when there is no directed edge?   
> 4. Why are the column sums reasonable dispite the obvious devision from aximonatic probability theory?      
> **End of exervise.**

> **Answers:**    

> 3. The damped transition matrix allows the random surfer to 'teleport' from any page to any other page with a small probability even when there is no directed edge by incorporating a damping factor. The damping factor introduces a small probability of jumping to any page in the graph, regardless of the presence of directed edges. This ensures that the random surfer can escape dead ends or spider traps and explore the entire graph more effectively.

> 4. The sums of the first five columns are 1.0, which is reasonable as it ensures that the probability distribution remains normalized. The last column sum of 0.15 indicates that there is a significant amount of probability mass being absorbed by the dead-end node, and makes sense as it happens to be (1-d), d being the damping factor of 0.85. In a perfect graph with no dead ends, all column sums should be 1.0 as per axiomatic probability theory.

## Hubs, Authorities, and the HITS Algorithm  

The hubs and authorities model is an alternative to PageRank. Rather than using a single metric to rank the importance of web pages, the **HITS** algorithm iteratively updates the **hub score** and **authority score** for each of the pages. 

The HITS algorithm updates the authority and hub scores iteratively. The authority score is sum of the hubs linked to it. This is computed by the matrix product of the association matrix and hubness vector: 
$$𝑎= \beta 𝐴 ℎ$$

Hub score is sum of the authorities it links to. The hub score (hubness) is compute by the matrix produce of the authority scores and the transpose of the association matrix: 
$$ℎ= \alpha 𝐴^𝑇 a$$

The algorithm iterates between updates to $𝑎$ and $ℎ$ until convergence. To ensure convergence, must normalize $𝑎$ and $ℎ$ to have unit Euclidean norm at each iteration. Therefore, the choice of $\alpha$ and $\beta$ are can therefore be set to a value of 1.0, and effectively ignored.       

> **Exercise 04-10:** To understand the HITS algorithm you will now create and test code for this algorithm. Follow these steps:  
> 1. Create a function called `HITS` with arguments of the association matrix, initial hub vector, initial authority vector, and the number of iterations of the algorithm to run. This function does the following inside a loop over the number of iterations:  
>    1. Updates the authority vector using the association matrix and the hub vector as argument to the `transition` function. 
>    2. Normalizes the authority vector by using `numpy.divide` with arguments of the updated authority vector and its L2 norm, computed with [numpy.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).  
>    3. Updates the hub vector using the transpose of the association matrix and the authority vector as argument to the `transition` function. 
>    4. Normalizes the hub vector by using `numpy.divide` with arguments of the updated hub vector and its L2 norm, computed with `numpy.linalg.norm`.  
> 2. The function returns both the hub and authority vectors
> 3. Initialize an initial hub and authority vector of length the dimension of the association matrix with uniformly distributed values of $\frac{1.0}{dimension(association\ matrix)}$.  
> 4. Display the resulting hub and authority vectors.  
> 5. Execute your function using the association matrix for the 6-page network and the initial hub and authority vectors as arguments. 

In [192]:
def HITS(association, hub, authority, iters=100):
    for _ in range(iters):
        # Update authority vector
        authority = np.dot(association.T, hub)
        # Normalize authority vector
        authority = np.divide(authority, np.linalg.norm(authority))
        
        # Update hub vector
        hub = np.dot(association, authority)
        # Normalize hub vector
        hub = np.divide(hub, np.linalg.norm(hub))
    return hub, authority

dimension = A_deadend.shape[0]
hub_start = np.ones(dimension) / dimension
initial_start = np.ones(dimension) / dimension

## Execute function
hubness, authority = HITS(A_deadend, hub_start, auth_start)
print(f"The hub ranks = {hubness}")
print(f"The authority ranks = {authority}")

The hub ranks = [0.51721531 0.5267355  0.56154289 0.28553154 0.24120396 0.        ]
The authority ranks = [0.29355748 0.18692427 0.18692427 0.66740602 0.49650196 0.38986875]


> Examine your results and answer the following questions:
> 1. Which three of the pages have the highest hub scores? Considering the graph of the pages, is this ordering consistent?  
> 2. Notice the last value of the hub scores. Is this value expected given the graph of the pages? 
> 3. Which three of the pages have the highest authority. Given the in degree of the pages is this ranking consistent?  
> 4. Compare the ranking of the pages based on authority that found with damped PageRank. Are these results consistent? 
> **End of exercise.**

> **Answers:**     
> 1. The three pages with the highest hub scores are pages 3,2,1 with 0.561, 0.526, 0.517 hub ranks respectivelu. This ordering is consistent with the graph of the pages, as these pages have the most outgoing links (3 each) and contribute most to the connectivity of the graph.

> 2. The last hub score is 0. This is expected because the corresponding page (Page 6) has no outgoing links, meaning it does not act as a hub and thus its hub score is zero.
> 
> 3. The three pages with the highest authority are those that are pointed to by other important pages in the graph, indicating their importance as authorities. Pages 4,5,6 have the highest authority scores at approx. 0.667, 0.496, 0.389 respsectively. This ranking is consistent with the graph, as these pages have the most links (2-3 each) coming from other important pages.

> 4. The authority rankings and PageRank rankings can be consistent but might differ due to the nature of the algorithms. PageRank considers the importance of incoming links and can distribute rank more evenly due to damping. In contrast, HITS distinctly separates hub and authority roles. Pages with high in-degree (receiving many links) will rank high in both HITS authority and PageRank, though specific values and rankings might slightly differ.


#### Copyright 2021, 2022, 2023, 2024 Stephen F Elston. All rights reserved.